<a href="https://www.kaggle.com/code/ahmed3okka/pg-s4-e9-automl-submission?scriptVersionId=195932138" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


In [2]:
from sklearn.model_selection import train_test_split,GridSearchCV,RepeatedStratifiedKFold, cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics

# Data preparation and data clearning

In [3]:
df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
df_sample = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
df.shape

(188533, 13)

In [5]:
def columns_info(data):
  cols = []
  dtypes = []
  unique_v = []
  n_unique_v = []
  null_count = []
  null_percent = []
      

  for col in data.columns:
    cols.append(col)
    dtypes.append(data[col].dtype)
    unique_v.append(data[col].unique())
    n_unique_v.append(data[col].nunique())
    null_count.append(data[col].isnull().sum())
    null_percent.append(round(data[col].isnull().sum()/data.shape[0],2))
    

  return pd.DataFrame({'cols':cols,'dtypes':dtypes,'unique_v':unique_v,'n_unique_v':n_unique_v,'sum_null':null_count,'%_null':null_percent})

In [6]:
columns_info(df)

,cols,dtypes,unique_v,n_unique_v,sum_null,%_null
0,id,int64,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",188533,0,0.00
1,brand,object,"[MINI, Lincoln, Chevrolet, Genesis, Mercedes-B...",57,0,0.00
2,model,object,"[Cooper S Base, LS V8, Silverado 2500 LT, G90 ...",1897,0,0.00
3,model_year,int64,"[2007, 2002, 2017, 2021, 2018, 2016, 2020, 201...",34,0,0.00
4,milage,int64,"[213000, 143250, 136731, 19500, 7388, 40950, 6...",6651,0,0.00
5,fuel_type,object,"[Gasoline, E85 Flex Fuel, nan, Hybrid, Diesel,...",7,5083,0.03
6,engine,object,"[172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,...",1117,0,0.00
7,transmission,object,"[A/T, Transmission w/Dual Shift Mode, 7-Speed ...",52,0,0.00
8,ext_col,object,"[Yellow, Silver, Blue, Black, White, Snowflake...",319,0,0.00
9,int_col,object,"[Gray, Beige, Black, –, Blue, White, Red, Brow...",156,0,0.00


In [7]:
columns_info(df_test)

,cols,dtypes,unique_v,n_unique_v,sum_null,%_null
0,id,int64,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",188533,0,0.00
1,brand,object,"[MINI, Lincoln, Chevrolet, Genesis, Mercedes-B...",57,0,0.00
2,model,object,"[Cooper S Base, LS V8, Silverado 2500 LT, G90 ...",1897,0,0.00
3,model_year,int64,"[2007, 2002, 2017, 2021, 2018, 2016, 2020, 201...",34,0,0.00
4,milage,int64,"[213000, 143250, 136731, 19500, 7388, 40950, 6...",6651,0,0.00
5,fuel_type,object,"[Gasoline, E85 Flex Fuel, nan, Hybrid, Diesel,...",7,5083,0.03
6,engine,object,"[172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,...",1117,0,0.00
7,transmission,object,"[A/T, Transmission w/Dual Shift Mode, 7-Speed ...",52,0,0.00
8,ext_col,object,"[Yellow, Silver, Blue, Black, White, Snowflake...",319,0,0.00
9,int_col,object,"[Gray, Beige, Black, –, Blue, White, Red, Brow...",156,0,0.00


In [8]:
df.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [9]:
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df = df.fillna(0)

In [10]:
columns_info(df)

,cols,dtypes,unique_v,n_unique_v,sum_null,%_null
0,id,int64,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",188533,0,0.0
1,brand,object,"[mini, lincoln, chevrolet, genesis, mercedes-b...",57,0,0.0
2,model,object,"[cooper_s_base, ls_v8, silverado_2500_lt, g90_...",1893,0,0.0
3,model_year,int64,"[2007, 2002, 2017, 2021, 2018, 2016, 2020, 201...",34,0,0.0
4,milage,int64,"[213000, 143250, 136731, 19500, 7388, 40950, 6...",6651,0,0.0
5,fuel_type,object,"[gasoline, e85_flex_fuel, 0, hybrid, diesel, p...",8,0,0.0
6,engine,object,"[172.0hp_1.6l_4_cylinder_engine_gasoline_fuel,...",1117,0,0.0
7,transmission,object,"[a/t, transmission_w/dual_shift_mode, 7-speed_...",51,0,0.0
8,ext_col,object,"[yellow, silver, blue, black, white, snowflake...",317,0,0.0
9,int_col,object,"[gray, beige, black, –, blue, white, red, brow...",152,0,0.0


In [11]:
x = df.drop(['price','id'],axis=1)
y = df['price']

In [12]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42 , stratify =y) 

In [13]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
train_dicts = x_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = x_test.to_dict(orient='records')
X_test = dv.transform(val_dicts)

In [14]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.46.0.4
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmposvc4l3c
  JVM stdout: /tmp/tmposvc4l3c/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmposvc4l3c/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,1 month and 30 days
H2O_cluster_name:,H2O_from_python_unknownUser_ao67q0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [15]:
%%time
train = h2o.import_file("../input/playground-series-s4e9/train.csv")
test = h2o.import_file("../input/playground-series-s4e9/test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
CPU times: user 218 ms, sys: 31 ms, total: 249 ms
Wall time: 6.56 s


In [16]:
x = train.columns
y = 'price'
x.remove(y)
x.remove('id')

In [17]:
aml = H2OAutoML(max_runtime_secs = 3500, seed = 42, project_name = "cars_price")
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |██
12:51:54.105: _train param, Dropping bad and constant columns: [clean_title]

████████████████████████████
13:02:15.901: XRT_1_AutoML_1_20240909_125118 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError

█
13:02:31.716: _train param, Dropping bad and constant columns: [clean_title]

████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),16/89
# GBM base models (used / total),7/35
# XGBoost base models (used / total),6/47
# DeepLearning base models (used / total),3/5
# DRF base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [18]:
lb = aml.leaderboard
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_4_AutoML_1_20240909_125118,72593.1,5.26976e+09,19211.3,0.536118,5.26976e+09
StackedEnsemble_AllModels_3_AutoML_1_20240909_125118,72595.5,5.27011e+09,19211,0.536438,5.27011e+09
StackedEnsemble_BestOfFamily_4_AutoML_1_20240909_125118,72604.9,5.27148e+09,19270.8,0.539621,5.27148e+09
StackedEnsemble_AllModels_2_AutoML_1_20240909_125118,72675.1,5.28167e+09,19308.2,0.541426,5.28167e+09
StackedEnsemble_BestOfFamily_3_AutoML_1_20240909_125118,72675.9,5.28179e+09,19313.2,0.541489,5.28179e+09
StackedEnsemble_BestOfFamily_5_AutoML_1_20240909_125118,72791.7,5.29863e+09,19383.7,0.57096,5.29863e+09
DeepLearning_1_AutoML_1_20240909_125118,72812.2,5.30162e+09,19743.6,0.553017,5.30162e+09
GBM_grid_1_AutoML_1_20240909_125118_model_7,72926.4,5.31826e+09,19666.4,nan,5.31826e+09
DeepLearning_grid_1_AutoML_1_20240909_125118_model_1,72953.1,5.32216e+09,19386.3,nan,5.32216e+09
GBM_grid_1_AutoML_1_20240909_125118_model_16,72960,5.32316e+09,19739.3,nan,5.32316e+09


In [19]:
# The leader model is stored here
aml.leader

key,value
Stacking strategy,cross_validation
Number of base models (used / total),16/89
# GBM base models (used / total),7/35
# XGBoost base models (used / total),6/47
# DeepLearning base models (used / total),3/5
# DRF base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [20]:
pred = aml.predict(test)
pred.head()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'model' has levels not trained on: ["allroad 2.7T"]
  warnings.warn(w)
/opt/conda/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'engine' has levels not trained on: ["177.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas"]
  warnings.warn(w)


predict
16312.3
76748.1
54923.4
23978.8
27769.8
15297.4
10648.3
41843.3
81666.1
29930.2


In [21]:
h2o.save_model(aml.leader, path = "./product_backorders_model_bin")

'/kaggle/working/product_backorders_model_bin/StackedEnsemble_AllModels_4_AutoML_1_20240909_125118'

In [22]:
sample_submission = pd.read_csv('../input/playground-series-s4e9/sample_submission.csv')
sample_submission.shape

(125690, 2)

In [23]:
sample_submission['price'] = pred.as_data_frame().values
sample_submission.to_csv('h2o_automl_submission.csv', index=False)

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [24]:
h2o.save_model(aml.leader, path = "submission1.csv")

'/kaggle/working/submission1.csv/StackedEnsemble_AllModels_4_AutoML_1_20240909_125118'

In [25]:
sample_submission.head()

,id,price
0,188533,16312.333467
1,188534,76748.146672
2,188535,54923.429846
3,188536,23978.788109
4,188537,27769.799741
